In [27]:
pip install mysql-connector-python openai pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
!pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable


In [35]:
from dotenv import load_dotenv
import os
import pandas as pd
import openai
import mysql.connector
load_dotenv("one.env")

# Get the API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

print("API Key Loaded Successfully (Hidden)")  


API Key Loaded Successfully (Hidden)


In [30]:

DB_CONFIG = {
    "host": "localhost",
    "user":"root",
    "password": "Zemoso@1234",
    "database": "company_db"
}



In [31]:
import openai
import json
import re

def refine_question(question):
    prompt = f"""
    Given the generic question: '{question}', refine it into a specific and detailed query about company data.

    The database schema:
    - Table: employees
    - Columns: employee_id, first_name, last_name, department, salary, hire_date

    Ensure the refined question only references these columns.

    Return a JSON object:
    - "refined_question": the improved question
    - "database_name": the targeted database

    Only return JSON, no explanations or formatting.
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=[{"role": "user", "content": prompt}]
    )

    try:
        return json.loads(response["choices"][0]["message"]["content"])
    except json.JSONDecodeError:
        return {"error": "Invalid JSON format"}


In [32]:
import openai
import re

def generate_sql_query(refined_question):
    prompt = f"""
    Convert the following refined question into an SQL query.

    The database schema:
    - Table: employees
    - Columns: employee_id, first_name, last_name, department, salary, hire_date

    Only use these columns in the query.

    Refined Question:
    '{refined_question}'

    Return **only** the SQL query, with no explanations or formatting.
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=[{"role": "user", "content": prompt}]
    )

    return response["choices"][0]["message"]["content"].strip()


In [33]:

# Function to execute the SQL query
def execute_query(query):
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute(query)
        rows = cursor.fetchall()

        # Fetch column names
        columns = [desc[0] for desc in cursor.description]

        # Convert to DataFrame for better visualization
        df = pd.DataFrame(rows, columns=columns)
        
        cursor.close()
        conn.close()

        return df

    except mysql.connector.Error as err:
        return f"Error: {err}"

In [36]:
# Main Execution Flow
if __name__ == "__main__":
    user_question = input("Enter a generic question about company data: ")
    
    refined_output = refine_question(user_question)
    print("\nRefined Question:", refined_output["refined_question"])

    sql_query = generate_sql_query(refined_output["refined_question"])
    print("\nGenerated SQL Query:\n", sql_query)

    print("\nExecuting Query...")
    result_df = execute_query(sql_query)
    print("\nQuery Result:\n", result_df)

Enter a generic question about company data: "What is the average salary in each department?"

Refined Question: What is the average salary for each department in the employees table?

Generated SQL Query:
 SELECT department, AVG(salary) AS average_salary
FROM employees
GROUP BY department;

Executing Query...

Query Result:
     department average_salary
0           HR   60000.000000
1  Engineering   90000.000000
2    Marketing   75000.000000
